In [2]:
import json
import requests
import urllib.parse
import pandas as pd 
import io
import re

import warnings                                                                 # Ignore warning related to pandas_profiling
warnings.filterwarnings('ignore') 

pd.set_option('display.max_columns', 100)

In [16]:
def get_exchange_rate(source, target = "EUR"):
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    source (string): string user to exchange
                    target (string): string user to exchange

            Returns:
                     ts(dataframe): data with given indentifer
   '''

  # Building blocks for the URL

  entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
  resource = 'data'           # The resource for data queries is always'data'
  flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
  key = 'M.'+ source + '.' + target + '.SP00.A'
  # key = 'Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N'    # Defining the dimension values, explained below

  # Define the parameters
  parameters = {'detail':"dataonly"}
  # Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
  request_url = entrypoint + resource + '/'+ flowRef + '/' + key

  # Make the HTTP request
  response = requests.get(request_url, params=parameters)

  response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
  df = pd.read_csv(io.StringIO(response.text))
  ts = df.filter(['TIME_PERIOD', 'OBS_VALUE',], axis=1)
  # ts['KEY'] = ts['KEY'].str[4:]
  # ts.rename(columns = {'KEY':'IDENTIFIER'}, inplace = True)
  return ts

  
exchange_data_frame =  get_exchange_rate('GBP')




In [17]:
def get_raw_data(identifier) :
  '''
    Returns the dataframe with the value of we sent as an indetifier
            Parameters:
                    identifier (string): key
                  

            Returns:
                     ts(dataframe): data with given indentifer
   '''

  # Building blocks for the URL
 
  entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
  resource = 'data'           # The resource for data queries is always'data'
  flowRef ='BP6'              # Dataflow describing the data that needs to be returned, exchange rates in this case
  key = identifier
  # key = 'Q.N.I8.W1.S1.S1.T.A.FA.D.F._Z.EUR._T._X.N'    # Defining the dimension values, explained below

  # Define the parameters
  parameters = {'detail':"dataonly"}
  # Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
  request_url = entrypoint + resource + '/'+ flowRef + '/' + key

  # Make the HTTP request
  response = requests.get(request_url, params=parameters)

  response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
  df = pd.read_csv(io.StringIO(response.text))
  
  ts = df.filter(['TIME_PERIOD', 'OBS_VALUE',], axis=1)

  return ts

  
raw_data_frame = get_raw_data("M.N.I8.W1.S1.S1.T.N.FA.F.F7.T.EUR._T.T.N")




In [22]:
def get_data(identifier, source= None):
  '''
    Returns the dataframe with the exchange rate data
            Parameters:
                    identifier (string): key
                    source (string) : currency
                  

            Returns:
                     final_df(dataframe): data with given indentifer
   '''



  raw_data_frame = get_raw_data(identifier)

  if source is None:
    return exchange_data_frame
  else:
    exchange_data_frame =  get_exchange_rate(source)
    megre_df = pd.merge(raw_data_frame,exchange_data_frame, on = "TIME_PERIOD", how = 'inner')
    megre_df['OBS_VALUE'] = megre_df['OBS_VALUE_x'] * megre_df['OBS_VALUE_y']
    final_df=megre_df.filter(['TIME_PERIOD','OBS_VALUE'])
    return final_df

get_data("M.N.I8.W1.S1.S1.T.N.FA.F.F7.T.EUR._T.T.N","GBP")

    TIME_PERIOD     OBS_VALUE
0       1999-01   1003.524746
1       1999-02    261.402398
2       1999-03  -1072.241252
3       1999-04  -2519.532218
4       1999-05   1112.665941
..          ...           ...
270     2021-07  18585.959501
271     2021-08  -7071.869956
272     2021-09    615.240309
273     2021-10   3888.862983
274     2021-11  18559.896198

[275 rows x 2 columns]
